# 03. Cohort & Retention 분석 실습

SQL로 직접 코호트 분석을 수행해봅니다.

**규칙:**
- 힌트는 최대한 안 보고 먼저 시도하세요
- 막히면 힌트를 펼쳐보세요
- 완료 후 `solution.sql`과 비교하세요

## 환경 설정

In [ ]:
import sqlite3
import pandas as pd
from pathlib import Path

# 데이터베이스 연결
DB_PATH = Path("../data/crm.db")
conn = sqlite3.connect(DB_PATH)

# SQL 실행 헬퍼 함수
def sql(query):
    """SQL 쿼리 실행 및 결과 반환"""
    return pd.read_sql(query, conn)

print("데이터베이스 연결 완료!")

---
## 준비: 데이터 파악

분석 전에 항상 데이터를 먼저 확인하세요.

In [ ]:
# 고객 가입일 분포 확인
sql("""
SELECT 
    strftime('%Y-%m', signup_date) as signup_month,
    COUNT(*) as customer_count
FROM customers
GROUP BY signup_month
ORDER BY signup_month
LIMIT 12
""")

In [ ]:
# 거래 데이터 기간 확인
sql("""
SELECT 
    MIN(transaction_date) as first_transaction,
    MAX(transaction_date) as last_transaction,
    COUNT(*) as total_transactions,
    COUNT(DISTINCT customer_id) as active_customers
FROM transactions
""")

---
# Mission 1: 코호트 정의

고객을 가입 월 기준으로 코호트로 분류해봅시다.

## Mission 1-1: 월별 코호트 생성

**문제:** 고객을 가입 월 기준으로 코호트를 정의하고, 각 코호트의 고객 수를 계산하세요.

출력 컬럼: `cohort_month`, `cohort_size`

In [ ]:
# TODO: 월별 코호트와 고객 수를 계산하는 SQL을 작성하세요
sql("""
SELECT 
    _____ as cohort_month,
    COUNT(*) as cohort_size
FROM customers
GROUP BY _____
ORDER BY cohort_month
""")

<details>
<summary>힌트 (클릭해서 펼치기)</summary>

- SQLite에서 날짜 포맷: `strftime('%Y-%m', signup_date)`
- 이렇게 하면 '2024-01' 형태로 연-월이 추출됩니다

</details>

## Mission 1-2: 고객별 코호트 할당

**문제:** 각 고객에게 코호트(가입 월)를 할당하세요.

출력 컬럼: `customer_id`, `signup_date`, `cohort_month`

In [ ]:
# TODO: 고객별 코호트 할당 SQL을 작성하세요
sql("""
SELECT 
    customer_id,
    signup_date,
    _____ as cohort_month
FROM customers
ORDER BY signup_date
LIMIT 10
""")

---
# Mission 2: 리텐션 계산

코호트별 리텐션을 계산해봅시다.

## Mission 2-1: 활동 월 계산

**문제:** 고객별로 활동(거래)이 있었던 월을 추출하세요.

출력 컬럼: `customer_id`, `activity_month`

In [ ]:
# TODO: 고객별 활동 월을 추출하는 SQL을 작성하세요
sql("""
SELECT DISTINCT
    customer_id,
    _____ as activity_month
FROM transactions
ORDER BY customer_id, activity_month
LIMIT 20
""")

<details>
<summary>힌트 (클릭해서 펼치기)</summary>

- `strftime('%Y-%m', transaction_date)` 사용
- DISTINCT로 중복 제거

</details>

## Mission 2-2: 가입 후 경과 기간 계산

**문제:** 각 활동에 대해 가입 후 몇 개월이 경과했는지 계산하세요.

```
경과 기간 = 활동 월 - 가입 월
```

출력 컬럼: `customer_id`, `cohort_month`, `activity_month`, `periods_since_signup`

In [ ]:
# TODO: 가입 후 경과 기간을 계산하는 SQL을 작성하세요
sql("""
WITH customer_cohort AS (
    SELECT 
        customer_id,
        strftime('%Y-%m', signup_date) as cohort_month
    FROM customers
),
customer_activity AS (
    SELECT DISTINCT
        customer_id,
        strftime('%Y-%m', transaction_date) as activity_month
    FROM transactions
)
SELECT 
    a.customer_id,
    c.cohort_month,
    a.activity_month,
    -- 경과 기간 계산 (년 차이 * 12 + 월 차이)
    (CAST(strftime('%Y', a.activity_month || '-01') AS INTEGER) - 
     CAST(strftime('%Y', c.cohort_month || '-01') AS INTEGER)) * 12 +
    (CAST(strftime('%m', a.activity_month || '-01') AS INTEGER) - 
     CAST(strftime('%m', c.cohort_month || '-01') AS INTEGER)) as periods_since_signup
FROM customer_activity a
JOIN customer_cohort c ON a.customer_id = c.customer_id
ORDER BY a.customer_id, a.activity_month
LIMIT 20
""")

## Mission 2-3: 코호트별 리텐션 매트릭스

**문제:** 코호트별, 기간별 리텐션을 계산하세요.

```
리텐션율 = (해당 기간 활성 고객 수 / 코호트 전체 고객 수) × 100
```

출력 컬럼: `cohort_month`, `period`, `active_customers`, `cohort_size`, `retention_rate`

In [ ]:
# TODO: 코호트별 리텐션 매트릭스를 계산하는 SQL을 작성하세요
sql("""
WITH customer_cohort AS (
    SELECT 
        customer_id,
        strftime('%Y-%m', signup_date) as cohort_month
    FROM customers
),
cohort_size AS (
    SELECT 
        cohort_month,
        COUNT(*) as size
    FROM customer_cohort
    GROUP BY cohort_month
),
customer_activity AS (
    SELECT DISTINCT
        t.customer_id,
        c.cohort_month,
        strftime('%Y-%m', t.transaction_date) as activity_month,
        (CAST(strftime('%Y', t.transaction_date) AS INTEGER) - 
         CAST(strftime('%Y', c.cohort_month || '-01') AS INTEGER)) * 12 +
        (CAST(strftime('%m', t.transaction_date) AS INTEGER) - 
         CAST(strftime('%m', c.cohort_month || '-01') AS INTEGER)) as period
    FROM transactions t
    JOIN customer_cohort c ON t.customer_id = c.customer_id
),
retention_data AS (
    SELECT 
        cohort_month,
        period,
        COUNT(DISTINCT customer_id) as active_customers
    FROM customer_activity
    WHERE period >= 0
    GROUP BY cohort_month, period
)
SELECT 
    r.cohort_month,
    r.period,
    r.active_customers,
    s.size as cohort_size,
    ROUND(_____ * 100, 2) as retention_rate
FROM retention_data r
JOIN cohort_size s ON r.cohort_month = s.cohort_month
WHERE r.period <= 6
ORDER BY r.cohort_month, r.period
""")

<details>
<summary>힌트 (클릭해서 펼치기)</summary>

- 리텐션율: `r.active_customers * 1.0 / s.size`
- period가 0이면 가입 월 (보통 100%)
- period가 1이면 가입 후 1개월

</details>

---
# Mission 3: 리텐션 분석

리텐션 데이터에서 인사이트를 도출해봅시다.

## Mission 3-1: 기간별 평균 리텐션

**문제:** 전체 코호트의 기간별 평균 리텐션을 계산하세요.

출력 컬럼: `period`, `avg_retention`, `min_retention`, `max_retention`

In [ ]:
# TODO: 기간별 평균 리텐션을 계산하는 SQL을 작성하세요
sql("""
WITH customer_cohort AS (
    SELECT 
        customer_id,
        strftime('%Y-%m', signup_date) as cohort_month
    FROM customers
),
cohort_size AS (
    SELECT cohort_month, COUNT(*) as size
    FROM customer_cohort
    GROUP BY cohort_month
),
customer_activity AS (
    SELECT DISTINCT
        t.customer_id,
        c.cohort_month,
        (CAST(strftime('%Y', t.transaction_date) AS INTEGER) - 
         CAST(strftime('%Y', c.cohort_month || '-01') AS INTEGER)) * 12 +
        (CAST(strftime('%m', t.transaction_date) AS INTEGER) - 
         CAST(strftime('%m', c.cohort_month || '-01') AS INTEGER)) as period
    FROM transactions t
    JOIN customer_cohort c ON t.customer_id = c.customer_id
),
cohort_retention AS (
    SELECT 
        a.cohort_month,
        a.period,
        COUNT(DISTINCT a.customer_id) * 100.0 / s.size as retention_rate
    FROM customer_activity a
    JOIN cohort_size s ON a.cohort_month = s.cohort_month
    WHERE a.period >= 0
    GROUP BY a.cohort_month, a.period, s.size
)
SELECT 
    period,
    ROUND(_____, 2) as avg_retention,
    ROUND(_____, 2) as min_retention,
    ROUND(_____, 2) as max_retention
FROM cohort_retention
WHERE period <= 6
GROUP BY period
ORDER BY period
""")

<details>
<summary>힌트 (클릭해서 펼치기)</summary>

- 평균: `AVG(retention_rate)`
- 최소: `MIN(retention_rate)`
- 최대: `MAX(retention_rate)`

</details>

## Mission 3-2: 최고/최저 성과 코호트 찾기

**문제:** 3개월 시점 리텐션이 가장 높은/낮은 코호트를 찾으세요.

**비즈니스 맥락:** 성과 차이의 원인을 분석하기 위함

In [ ]:
# TODO: 3개월 리텐션 기준 최고/최저 코호트를 찾는 SQL을 작성하세요
sql("""
WITH customer_cohort AS (
    SELECT 
        customer_id,
        strftime('%Y-%m', signup_date) as cohort_month
    FROM customers
),
cohort_size AS (
    SELECT cohort_month, COUNT(*) as size
    FROM customer_cohort
    GROUP BY cohort_month
),
month3_retention AS (
    SELECT 
        c.cohort_month,
        COUNT(DISTINCT t.customer_id) * 100.0 / s.size as retention_rate,
        s.size as cohort_size
    FROM customer_cohort c
    JOIN cohort_size s ON c.cohort_month = s.cohort_month
    LEFT JOIN transactions t ON c.customer_id = t.customer_id
        AND (CAST(strftime('%Y', t.transaction_date) AS INTEGER) - 
             CAST(strftime('%Y', c.cohort_month || '-01') AS INTEGER)) * 12 +
            (CAST(strftime('%m', t.transaction_date) AS INTEGER) - 
             CAST(strftime('%m', c.cohort_month || '-01') AS INTEGER)) = 3
    GROUP BY c.cohort_month, s.size
)
SELECT 
    cohort_month,
    cohort_size,
    ROUND(retention_rate, 2) as month3_retention,
    CASE 
        WHEN retention_rate = (SELECT MAX(retention_rate) FROM month3_retention) THEN 'BEST'
        WHEN retention_rate = (SELECT MIN(retention_rate) FROM month3_retention) THEN 'WORST'
        ELSE ''
    END as ranking
FROM month3_retention
ORDER BY retention_rate DESC
""")

**생각해보기:**
- 최고 성과 코호트가 가입한 시점에 어떤 특별한 이벤트가 있었을까?
- 최저 성과 코호트의 문제는 무엇이었을까?

---
# Mission 4: 세그먼트별 리텐션

채널별로 리텐션 차이가 있는지 분석해봅시다.

## Mission 4-1: 채널별 3개월 리텐션

**문제:** 획득 채널별로 3개월 리텐션을 비교하세요.

출력 컬럼: `channel`, `total_customers`, `retained_at_month3`, `retention_rate`

In [ ]:
# TODO: 채널별 3개월 리텐션을 계산하는 SQL을 작성하세요
sql("""
WITH customer_info AS (
    SELECT 
        customer_id,
        acquisition_channel,
        strftime('%Y-%m', signup_date) as cohort_month
    FROM customers
),
month3_activity AS (
    SELECT DISTINCT
        c.customer_id,
        c.acquisition_channel
    FROM customer_info c
    JOIN transactions t ON c.customer_id = t.customer_id
    WHERE (CAST(strftime('%Y', t.transaction_date) AS INTEGER) - 
           CAST(strftime('%Y', c.cohort_month || '-01') AS INTEGER)) * 12 +
          (CAST(strftime('%m', t.transaction_date) AS INTEGER) - 
           CAST(strftime('%m', c.cohort_month || '-01') AS INTEGER)) = 3
)
SELECT 
    c.acquisition_channel as channel,
    COUNT(DISTINCT c.customer_id) as total_customers,
    COUNT(DISTINCT a.customer_id) as retained_at_month3,
    ROUND(_____ * 100, 2) as retention_rate
FROM customer_info c
LEFT JOIN month3_activity a ON c.customer_id = a.customer_id
GROUP BY c.acquisition_channel
ORDER BY retention_rate DESC
""")

<details>
<summary>힌트 (클릭해서 펼치기)</summary>

- 리텐션율: `COUNT(DISTINCT a.customer_id) * 1.0 / COUNT(DISTINCT c.customer_id)`
- LEFT JOIN을 사용해야 활동이 없는 고객도 분모에 포함됩니다

</details>

---
# Mission 5: 실무 시나리오

**시나리오:**

> 프로덕트팀에서 리텐션 개선 전략을 수립하려고 합니다.
> 코호트 분석 결과를 바탕으로 다음을 도출하세요:
>
> 1. 리텐션이 급격히 떨어지는 시점 (이탈이 가장 많은 기간)
> 2. 가장 리텐션이 좋은/나쁜 채널
> 3. 리텐션 개선을 위한 권고안

**당신의 분석을 아래에 작성하세요:**

In [ ]:
# 자유롭게 분석 쿼리를 작성하세요

# 1. 기간별 이탈 분석 (어느 시점에 가장 많이 이탈하는지)
sql("""
-- 여기에 쿼리 작성

""")

In [ ]:
# 2. 채널별 리텐션 비교
sql("""
-- 여기에 쿼리 작성

""")

### 나의 분석 결과 및 권고안

(여기에 분석 결과를 작성하세요)

1. **이탈이 가장 많은 시점**: 
2. **채널별 리텐션 차이**: 
3. **권고안**: 

---
# 회고

실습을 마치며 아래 질문에 답해보세요.

1. **어떤 부분이 어려웠나요?**

2. **새롭게 배운 SQL 기법은?**

3. **코호트 분석을 실무에서 어떻게 활용하겠습니까?**

4. **면접에서 이 내용을 어떻게 설명하겠습니까?**

In [ ]:
# 데이터베이스 연결 종료
conn.close()
print("실습 완료!")